# Data Consolidation

Read data from `data/raw` folder and consolidates it in a single file dataset in `data/consolidated` folder.

Might take some time to run.


## Load Data

In [ ]:
import pandas as pd
from pathlib import Path

# Path to the root dataset folder
DATASET_DIR = Path("../data/raw")

# Collect all XLSX files recursively
xlsx_files = list(DATASET_DIR.rglob("*.xlsx"))

all_dfs = []

for i,file in enumerate(xlsx_files):
    print(i, "/", len(xlsx_files), ":", file)
    # Extract metadata from path
    subject = file.parts[-3]   # first folder name
    label = file.parts[-2]     # second folder name
    experiment = file.stem     # file name without extension
    
    # Load xlsx into DataFrame
    df = pd.read_excel(file, header=0)  # assumes first row is header
    
    # Add metadata columns
    df["subject"] = subject
    df["label"] = label
    df["experiment"] = experiment
    
    all_dfs.append(df)

# Combine everything into a single DataFrame
dataset_df = pd.concat(all_dfs, ignore_index=True)

# Quick check
print(dataset_df.shape)


In [ ]:
# Check Col Types
exclude_cols = ["timestamp", "subject", "label", "experiment"]

for col in dataset_df.columns:
    if col not in exclude_cols:
        dataset_df[col] = dataset_df[col].astype(float)

print(dataset_df.dtypes)


In [ ]:
# Check data
dataset_df

## Check for missing values

In [ ]:
# Count NaN / missing values per column
missing_counts = dataset_df.isna().sum()

# Percentage of missing values per column
missing_percent = (missing_counts / len(dataset_df)) * 100

# Combine into a summary DataFrame
missing_report = pd.DataFrame({
    "missing_count": missing_counts,
    "missing_percent": missing_percent
}).sort_values(by="missing_count", ascending=False)

# Total missing values across the whole dataset
total_missing = dataset_df.isna().sum().sum()
total_cells = dataset_df.shape[0] * dataset_df.shape[1]
total_missing_percent = (total_missing / total_cells) * 100

print(missing_report)
print("\n--- Overall Missing Values ---")
print(f"Total missing values: {total_missing}")
print(f"Total cells: {total_cells}")
print(f"Overall missing percent: {total_missing_percent:.4f}%")


## Threat Missing values

No need, no missing values found.

## Drop Duplicates

In [ ]:
# Save original shape
original_shape = dataset_df.shape

# Drop duplicates
dataset_df = dataset_df.drop_duplicates().reset_index(drop=True)

# New shape
new_shape = dataset_df.shape

# Report
dropped = original_shape[0] - new_shape[0]
print(f"Original shape: {original_shape}")
print(f"New shape: {new_shape}")
print(f"Duplicates dropped: {dropped}")


## Export As Feather

In [ ]:
# Save the cleaned DataFrame as Parquet
dataset_df.to_parquet(
    Path("../data/consolidated/fall_dataset.parquet"),
    index=False,              
    engine="pyarrow",         
    compression="snappy"      
)

In [ ]:
# Reload test
df_test = pd.read_parquet(Path("../data/consolidated/fall_dataset.parquet"))
df_test